## Tensor Nuclear Norm minimization with Discrete Cosine Transform (TNN-DCT)

This notebook shows how to implement a TNN-DCT imputer on some real-world data sets. For an in-depth discussion of TNN-DCT, please see [1].

<div class="alert alert-block alert-info">
<font color="black">
<b>[1]</b> Canyi Lu, Xi Peng, Yunchao Wei (2019). <b>Low-rank Tensor Completion with a New Tensor Nuclear Norm Induced by Invertible Linear Transforms</b>. CVPR 2019 <a href="https://openaccess.thecvf.com/content_CVPR_2019/papers/Lu_Low-Rank_Tensor_Completion_With_a_New_Tensor_Nuclear_Norm_Induced_CVPR_2019_paper.pdf" title="PDF"><b>[PDF]</b></a> 
</font>
</div>


In [1]:
import numpy as np
from numpy.linalg import inv as inv
import matplotlib.pyplot as plt

%matplotlib inline

### Define kernels

We start by introducing some necessary functions that relies on `Numpy`.


In [2]:
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

In [3]:
from scipy.fftpack import dctn, idctn

def tsvt_tnn(tensor, tau):
    dim = tensor.shape
    X = np.zeros(dim)
    tensor = dctn(tensor, axes = (2, ), norm = 'ortho')
    for t in range(dim[2]):
        u, s, v = np.linalg.svd(tensor[:, :, t], full_matrices = False)
        s = s - tau
        s[s < 0] = 0
        X[:, :, t] = u @ np.diag(s) @ v
    return idctn(X, axes = (2, ), norm = 'ortho')

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>compute_mape</code>:</b> <font color="black">Compute the value of Mean Absolute Percentage Error (MAPE).</font></li>
<li><b><code>compute_rmse</code>:</b> <font color="black">Compute the value of Root Mean Square Error (RMSE).</font></li>
</ul>
</div>

> Note that $$\mathrm{MAPE}=\frac{1}{n} \sum_{i=1}^{n} \frac{\left|y_{i}-\hat{y}_{i}\right|}{y_{i}} \times 100, \quad\mathrm{RMSE}=\sqrt{\frac{1}{n} \sum_{i=1}^{n}\left(y_{i}-\hat{y}_{i}\right)^{2}},$$ where $n$ is the total number of estimated values, and $y_i$ and $\hat{y}_i$ are the actual value and its estimation, respectively.

In [4]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

The main idea behind TNN-DCT is to implement tensor singular value thresholding on discrete cosine transformed data.

In [5]:
def imputer(dense_tensor, sparse_tensor, rho0, epsilon, maxiter):
    """TNN-DCT-imputer."""
    dim = np.array(sparse_tensor.shape)
    pos_missing = np.where(sparse_tensor == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))

    T = np.zeros(dim)                         # \boldsymbol{\mathcal{T}}
    Z = sparse_tensor.copy()                     # \boldsymbol{Z}
    Z[pos_missing] = np.mean(sparse_tensor[sparse_tensor != 0])
    it = 0
    last_ten = sparse_tensor.copy()
    snorm = np.linalg.norm(sparse_tensor)
    rho = rho0
    while True:
        rho = min(rho * 1.05, 1e5)
        X = tsvt_tnn(Z - T / rho, 1 / rho)
        temp2 = rho * X + T
        Z[pos_missing] = temp2[pos_missing] / rho
        T = T + rho * (X - Z)
        tol = np.linalg.norm((X - last_ten)) / snorm
        last_ten = X.copy()
        it += 1
        if it % 1 == 0:
            print('Iter: {}'.format(it))
            print('Tolerance: {:.6}'.format(tol))
            print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], X[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], X[pos_test])))
            print()
        if (tol < epsilon) or (it >= maxiter):
            break

    print('Total iteration: {}'.format(it))
    print('Tolerance: {:.6}'.format(tol))
    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], X[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], X[pos_test])))
    print()

    return X

### PeMS-4W

We generate **random missing (RM)** values on California traffic speed data set.

In [6]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 4 * 7)

missing_rate = 0.3

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor

In [7]:
import time
start = time.time()
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.647975
MAPE: 0.0944086
RMSE: 6.5952

Iter: 2
Tolerance: 0.0624977
MAPE: 0.0659597
RMSE: 5.2035

Iter: 3
Tolerance: 0.0417232
MAPE: 0.0547732
RMSE: 4.58947

Iter: 4
Tolerance: 0.0314865
MAPE: 0.047675
RMSE: 4.08114

Iter: 5
Tolerance: 0.0251173
MAPE: 0.0427768
RMSE: 3.73007

Iter: 6
Tolerance: 0.0208854
MAPE: 0.0390501
RMSE: 3.46187

Iter: 7
Tolerance: 0.0187556
MAPE: 0.0359018
RMSE: 3.22428

Iter: 8
Tolerance: 0.0166505
MAPE: 0.0332504
RMSE: 3.01977

Iter: 9
Tolerance: 0.0156447
MAPE: 0.0307487
RMSE: 2.82316

Iter: 10
Tolerance: 0.014577
MAPE: 0.0285299
RMSE: 2.64417

Iter: 11
Tolerance: 0.0129226
MAPE: 0.0266966
RMSE: 2.49391

Iter: 12
Tolerance: 0.0116454
MAPE: 0.0251396
RMSE: 2.3656

Iter: 13
Tolerance: 0.0101913
MAPE: 0.0239068
RMSE: 2.26028

Iter: 14
Tolerance: 0.00911342
MAPE: 0.0228929
RMSE: 2.17082

Iter: 15
Tolerance: 0.00835471
MAPE: 0.02198
RMSE: 2.08913

Iter: 16
Tolerance: 0.00796722
MAPE: 0.0211518
RMSE: 2.01355

Iter: 17
Tolerance: 0.00731944
MAPE: 0

In [8]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 4 * 7)

missing_rate = 0.7

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor

In [9]:
import time
start = time.time()
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 1.49571
MAPE: 0.113089
RMSE: 7.76575

Iter: 2
Tolerance: 0.0606419
MAPE: 0.0994213
RMSE: 6.82448

Iter: 3
Tolerance: 0.0647124
MAPE: 0.0821937
RMSE: 6.00534

Iter: 4
Tolerance: 0.058776
MAPE: 0.0696222
RMSE: 5.51436

Iter: 5
Tolerance: 0.0489284
MAPE: 0.0637208
RMSE: 5.22021

Iter: 6
Tolerance: 0.0399298
MAPE: 0.0608704
RMSE: 4.987

Iter: 7
Tolerance: 0.0358055
MAPE: 0.0568874
RMSE: 4.67871

Iter: 8
Tolerance: 0.0350723
MAPE: 0.0517904
RMSE: 4.34389

Iter: 9
Tolerance: 0.03288
MAPE: 0.0480133
RMSE: 4.09141

Iter: 10
Tolerance: 0.0289579
MAPE: 0.0458073
RMSE: 3.90971

Iter: 11
Tolerance: 0.0254097
MAPE: 0.0440933
RMSE: 3.75971

Iter: 12
Tolerance: 0.0231204
MAPE: 0.0420513
RMSE: 3.60634

Iter: 13
Tolerance: 0.0220689
MAPE: 0.0397162
RMSE: 3.45276

Iter: 14
Tolerance: 0.0209427
MAPE: 0.0375499
RMSE: 3.31475

Iter: 15
Tolerance: 0.0196128
MAPE: 0.0356901
RMSE: 3.18927

Iter: 16
Tolerance: 0.0184424
MAPE: 0.034039
RMSE: 3.06994

Iter: 17
Tolerance: 0.0174981
MAPE: 0.0324

We generate **non-random missing (NM)** values. Then, we conduct the imputation experiment.

In [10]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

In [11]:
import time
start = time.time()
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.646364
MAPE: 0.0962517
RMSE: 6.69797

Iter: 2
Tolerance: 0.0634503
MAPE: 0.071281
RMSE: 5.50369

Iter: 3
Tolerance: 0.0407312
MAPE: 0.0626559
RMSE: 5.14167

Iter: 4
Tolerance: 0.0259223
MAPE: 0.0600005
RMSE: 4.99106

Iter: 5
Tolerance: 0.021182
MAPE: 0.0595976
RMSE: 4.91881

Iter: 6
Tolerance: 0.0190736
MAPE: 0.0604349
RMSE: 4.91262

Iter: 7
Tolerance: 0.0158726
MAPE: 0.0614999
RMSE: 4.93633

Iter: 8
Tolerance: 0.0151888
MAPE: 0.0621191
RMSE: 4.95309

Iter: 9
Tolerance: 0.0135718
MAPE: 0.0624342
RMSE: 4.96291

Iter: 10
Tolerance: 0.0120054
MAPE: 0.0625497
RMSE: 4.96541

Iter: 11
Tolerance: 0.0103608
MAPE: 0.062577
RMSE: 4.96505

Iter: 12
Tolerance: 0.00906152
MAPE: 0.0625479
RMSE: 4.96248

Iter: 13
Tolerance: 0.00823469
MAPE: 0.0625004
RMSE: 4.95925

Iter: 14
Tolerance: 0.0078163
MAPE: 0.0624584
RMSE: 4.95637

Iter: 15
Tolerance: 0.00740875
MAPE: 0.0624265
RMSE: 4.95431

Iter: 16
Tolerance: 0.00667963
MAPE: 0.0624042
RMSE: 4.95255

Iter: 17
Tolerance: 0.00614749
MA

In [12]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

In [13]:
import time
start = time.time()
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 1.4926
MAPE: 0.113239
RMSE: 7.77958

Iter: 2
Tolerance: 0.0636587
MAPE: 0.0999848
RMSE: 6.87108

Iter: 3
Tolerance: 0.0713378
MAPE: 0.0866489
RMSE: 6.23801

Iter: 4
Tolerance: 0.0482445
MAPE: 0.078834
RMSE: 5.95812

Iter: 5
Tolerance: 0.0323238
MAPE: 0.0748084
RMSE: 5.82881

Iter: 6
Tolerance: 0.0247181
MAPE: 0.0722041
RMSE: 5.73153

Iter: 7
Tolerance: 0.0233185
MAPE: 0.0705514
RMSE: 5.6494

Iter: 8
Tolerance: 0.021234
MAPE: 0.0699221
RMSE: 5.5888

Iter: 9
Tolerance: 0.0197154
MAPE: 0.0702692
RMSE: 5.5604

Iter: 10
Tolerance: 0.0184782
MAPE: 0.0711681
RMSE: 5.55905

Iter: 11
Tolerance: 0.0164746
MAPE: 0.0722248
RMSE: 5.57531

Iter: 12
Tolerance: 0.0149444
MAPE: 0.0731311
RMSE: 5.5977

Iter: 13
Tolerance: 0.013018
MAPE: 0.0737605
RMSE: 5.61773

Iter: 14
Tolerance: 0.0118181
MAPE: 0.0741067
RMSE: 5.63111

Iter: 15
Tolerance: 0.0108466
MAPE: 0.0742336
RMSE: 5.63724

Iter: 16
Tolerance: 0.00988963
MAPE: 0.074212
RMSE: 5.63726

Iter: 17
Tolerance: 0.00885919
MAPE: 0.07411

### PeMS-8W

We generate **random missing (RM)** values on California traffic speed data set.

In [14]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 8 * 7)

missing_rate = 0.3

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor

In [15]:
import time
start = time.time()
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.650439
MAPE: 0.0922593
RMSE: 6.47367

Iter: 2
Tolerance: 0.066228
MAPE: 0.0625579
RMSE: 5.06299

Iter: 3
Tolerance: 0.0383187
MAPE: 0.0537593
RMSE: 4.57595

Iter: 4
Tolerance: 0.0273082
MAPE: 0.0482377
RMSE: 4.17592

Iter: 5
Tolerance: 0.0259691
MAPE: 0.0430478
RMSE: 3.78784

Iter: 6
Tolerance: 0.0210778
MAPE: 0.0397567
RMSE: 3.53304

Iter: 7
Tolerance: 0.018719
MAPE: 0.0365491
RMSE: 3.29057

Iter: 8
Tolerance: 0.0173685
MAPE: 0.0336829
RMSE: 3.07368

Iter: 9
Tolerance: 0.0157692
MAPE: 0.031254
RMSE: 2.87807

Iter: 10
Tolerance: 0.0149143
MAPE: 0.0290234
RMSE: 2.69315

Iter: 11
Tolerance: 0.0131924
MAPE: 0.0272058
RMSE: 2.53953

Iter: 12
Tolerance: 0.0118896
MAPE: 0.0256438
RMSE: 2.40734

Iter: 13
Tolerance: 0.0105773
MAPE: 0.024322
RMSE: 2.29451

Iter: 14
Tolerance: 0.00952487
MAPE: 0.0232427
RMSE: 2.19914

Iter: 15
Tolerance: 0.00858261
MAPE: 0.0223277
RMSE: 2.11583

Iter: 16
Tolerance: 0.00800938
MAPE: 0.021492
RMSE: 2.03987

Iter: 17
Tolerance: 0.00748502
MAPE:

In [16]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 8 * 7)

missing_rate = 0.7

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor

In [17]:
import time
start = time.time()
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 1.50181
MAPE: 0.115507
RMSE: 7.90727

Iter: 2
Tolerance: 0.0668892
MAPE: 0.0967279
RMSE: 6.68787

Iter: 3
Tolerance: 0.0713258
MAPE: 0.076233
RMSE: 5.73577

Iter: 4
Tolerance: 0.0615519
MAPE: 0.0654057
RMSE: 5.333

Iter: 5
Tolerance: 0.0456658
MAPE: 0.0632323
RMSE: 5.2252

Iter: 6
Tolerance: 0.0351923
MAPE: 0.0608785
RMSE: 5.02298

Iter: 7
Tolerance: 0.0336892
MAPE: 0.0563504
RMSE: 4.70967

Iter: 8
Tolerance: 0.0341277
MAPE: 0.0513628
RMSE: 4.38522

Iter: 9
Tolerance: 0.0325337
MAPE: 0.0480522
RMSE: 4.12751

Iter: 10
Tolerance: 0.0292369
MAPE: 0.0463485
RMSE: 3.94916

Iter: 11
Tolerance: 0.0253781
MAPE: 0.0449474
RMSE: 3.81361

Iter: 12
Tolerance: 0.0227708
MAPE: 0.0429179
RMSE: 3.67203

Iter: 13
Tolerance: 0.0218922
MAPE: 0.0403492
RMSE: 3.51456

Iter: 14
Tolerance: 0.0213635
MAPE: 0.0378975
RMSE: 3.3633

Iter: 15
Tolerance: 0.0202517
MAPE: 0.0359412
RMSE: 3.23233

Iter: 16
Tolerance: 0.0188343
MAPE: 0.0343724
RMSE: 3.11519

Iter: 17
Tolerance: 0.0177497
MAPE: 0.032

We generate **non-random missing (NM)** values. Then, we conduct the imputation experiment.

In [18]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

In [19]:
import time
start = time.time()
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.648323
MAPE: 0.0936155
RMSE: 6.55669

Iter: 2
Tolerance: 0.0662785
MAPE: 0.0658574
RMSE: 5.25448

Iter: 3
Tolerance: 0.0379932
MAPE: 0.058295
RMSE: 4.93551

Iter: 4
Tolerance: 0.0264553
MAPE: 0.0555825
RMSE: 4.78207

Iter: 5
Tolerance: 0.0225048
MAPE: 0.0547291
RMSE: 4.69314

Iter: 6
Tolerance: 0.0190764
MAPE: 0.0555272
RMSE: 4.68328

Iter: 7
Tolerance: 0.01687
MAPE: 0.0564281
RMSE: 4.69665

Iter: 8
Tolerance: 0.0155852
MAPE: 0.0569633
RMSE: 4.70909

Iter: 9
Tolerance: 0.0140113
MAPE: 0.0572378
RMSE: 4.71634

Iter: 10
Tolerance: 0.0122761
MAPE: 0.0573282
RMSE: 4.71736

Iter: 11
Tolerance: 0.0107037
MAPE: 0.0573281
RMSE: 4.71538

Iter: 12
Tolerance: 0.00964381
MAPE: 0.057276
RMSE: 4.712

Iter: 13
Tolerance: 0.00848572
MAPE: 0.0572225
RMSE: 4.70901

Iter: 14
Tolerance: 0.00804687
MAPE: 0.05719
RMSE: 4.70708

Iter: 15
Tolerance: 0.00734762
MAPE: 0.057151
RMSE: 4.70456

Iter: 16
Tolerance: 0.00670407
MAPE: 0.0571273
RMSE: 4.70283

Iter: 17
Tolerance: 0.00639745
MAPE: 0

In [20]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

In [21]:
import time
start = time.time()
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 1.50017
MAPE: 0.115751
RMSE: 7.91847

Iter: 2
Tolerance: 0.0690349
MAPE: 0.0970697
RMSE: 6.7134

Iter: 3
Tolerance: 0.0744922
MAPE: 0.0795763
RMSE: 5.8899

Iter: 4
Tolerance: 0.0510859
MAPE: 0.0704942
RMSE: 5.55476

Iter: 5
Tolerance: 0.0360041
MAPE: 0.0671557
RMSE: 5.44755

Iter: 6
Tolerance: 0.0257861
MAPE: 0.0647772
RMSE: 5.35043

Iter: 7
Tolerance: 0.024361
MAPE: 0.0626775
RMSE: 5.24056

Iter: 8
Tolerance: 0.0231581
MAPE: 0.0616292
RMSE: 5.15438

Iter: 9
Tolerance: 0.0221646
MAPE: 0.062035
RMSE: 5.11825

Iter: 10
Tolerance: 0.0198435
MAPE: 0.0633434
RMSE: 5.12545

Iter: 11
Tolerance: 0.0173334
MAPE: 0.0647381
RMSE: 5.15321

Iter: 12
Tolerance: 0.0155331
MAPE: 0.0657754
RMSE: 5.18233

Iter: 13
Tolerance: 0.0139495
MAPE: 0.0663774
RMSE: 5.20395

Iter: 14
Tolerance: 0.0128522
MAPE: 0.0666335
RMSE: 5.21623

Iter: 15
Tolerance: 0.01136
MAPE: 0.066665
RMSE: 5.22066

Iter: 16
Tolerance: 0.0103345
MAPE: 0.0665617
RMSE: 5.21902

Iter: 17
Tolerance: 0.0093323
MAPE: 0.06639

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>